In [1]:
from functools import wraps
from transformers import AutoTokenizer, pipeline
import torch

device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'
cuda_v = torch.version.cuda if device != "cpu" else ""
f"Running on device: '{device}' with CUDA version '{cuda_v}."

"Running on device: 'cuda:0' with CUDA version '11.8."

In [2]:
MODEL_BASE = "tiiuae/falcon-7b"
MODEL_INSTRUCT = "tiiuae/falcon-7b-instruct"
model = MODEL_INSTRUCT # MODEL_INSTRUCT  
#model = "tiiuae/falcon-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model)

In [3]:
# update tokenizer to not return type ids
#  see: https://huggingface.co/tiiuae/falcon-7b-instruct/discussions/2
org_call_one = tokenizer._call_one

@wraps(org_call_one)
def _call_one_wrapped(*x, **y):
    y['return_token_type_ids'] = False
    return org_call_one(*x, **y)

tokenizer._call_one = _call_one_wrapped

In [4]:
falcon = pipeline(
    model=model,
    tokenizer=tokenizer,
    max_length=50,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    pad_token_id=tokenizer.eos_token_id,
    return_full_text=False # Ten fragment powoduje, że nie doklejamy promptu na początku odpowiedzi
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [5]:
from IPython.display import display, HTML

def pretty_print(text):
    return display(HTML('<font size="6">' + text.replace("\\n","<br>") + '</font>'))

In [6]:
text = falcon(['Who won the last FIFA World Cup?'])
pretty_print(text[0][0]['generated_text'])

c:\Users\krzysztof.joachimiak\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


In [43]:
text[0][0]['generated_text'][221]

'o'

In [10]:
falcon(['Jaka jest najdłuższa rzeka w Polsce?'])

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


[[{'generated_text': 'Jaka jest najdłuższa rzeka w Polsce?\nW Polsce jest wiele rzek, które są bardzo długie. Wśród nich najdłuższa jest rzeka Dunaj.\n'}]]